In [1]:
import tensorflow as tf
from tensorflow import keras
import segmentation_models as sm
import os
import cv2
import numpy as np
from lib.predict_image import *
from lib.json_name_path import Jsonname    # #取得json檔案的位置and fullname modify by charley 1120117


ckptname='test'  #最佳CKPT編號 1110830
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sm.set_framework('tf.keras')
sm.framework()
print("TF version: {}.".format(tf.__version__))
print("Keras version: {}.".format(keras.__version__))
            
def solve_cudnn_error():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
          
           
solve_cudnn_error()



Segmentation Models: using `tf.keras` framework.
TF version: 2.2.0.
Keras version: 2.3.0-tf.
1 Physical GPUs, 1 Logical GPUs


In [2]:
# pip install opencv-python


In [3]:
#import os
#Add  by charley 配合使用 使 os.getcwd() 可正確顯示
print('change directory 配合使用使 os.getcwd() 可正確顯示Start')
#os.chdir(os.path.dirname(os.path.abspath(__file__)))  #ipynb 不可用
from pathlib import Path  #改為目前的資料夾引用物件
os.chdir(Path.cwd())    #改為目前的資料夾*********
cwd = os.getcwd()
print(cwd)
print('change directory 配合使用使 os.getcwd() 可正確顯示End')
#print(cwd)
AI_Path = cwd.split('/C_pred')[0]
print(AI_Path)
# AI_Data_jsonFile=AI_Path+'/B_AI_training/Dentistry_config.json'  1120117

change directory 配合使用使 os.getcwd() 可正確顯示Start
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/C_pred
change directory 配合使用使 os.getcwd() 可正確顯示End
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet


In [4]:
# import json
# # jsonFile= = r'F:\SurgeryAnalytics\AI_Cases\Dentistry\1_Linknet_efficientnetb7_0_3\B_AI_training\Dentistry_config.json'
# jsonFile=AI_Data_jsonFile
# with open(jsonFile,"r") as f:
#     config = json.load(f)

In [5]:
AI_Data_jsonFile = Jsonname()
print(AI_Data_jsonFile)   #注意等等跑AI需要過來複製一下路徑
with open(AI_Data_jsonFile,"r") as f:
     config = json.load(f)

AI_Data_jsonFile : == /media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/Dentistry_AI_ZERO_New_1120808_500_linknet_config.json
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/Dentistry_AI_ZERO_New_1120808_500_linknet_config.json


In [6]:
config

{'surgeryName': 'Dentistry',
 'label_folder_path': '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/0_ori',
 'trans_folder_path': '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/1_trans',
 'split_folder_path': '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/2_split',
 'correct_label': ['Alveolar_bone',
  'Caries',
  'Crown',
  'Dentin',
  'Enamel',
  'Implant',
  'Mandibular_alveolar_nerve',
  'Maxillary_sinus',
  'Periapical_lesion',
  'Post_and_core',
  'Pulp',
  'Restoration',
  'Root_canal_filling'],
 'detect_label_list': ['Alveolar_bone',
  'Caries',
  'Crown',
  'Dentin',
  'Enamel',
  'Implant',
  'Mandibular_alveolar_nerve',
  'Maxillary_sinus',
  'Periapical_lesion',
  'Post_and_core',
  'Pulp',
  'Restoration',
  'Root_canal_filling'],
 'Data_total_excel': '/media/

# 參數設定
## [Model ref](https://github.com/qubvel/segmentation_models)

In [7]:
import os
import time
time.localtime()
train_time=time.strftime("%Y%m%d%H%M%S", time.localtime())

print('change directory 配合使用使 os.getcwd() 可正確顯示Start')
#os.chdir(os.path.dirname(os.path.abspath(__file__)))  #ipynb 不可用
from pathlib import Path  #改為目前的資料夾引用物件
os.chdir(Path.cwd())    #改為目前的資料夾*********
cwd = os.getcwd()
print(cwd)
print('change directory 配合使用使 os.getcwd() 可正確顯示End')

DATA_DIR = config["split_folder_path"]# 轉好檔的標記資料路徑(label為mask(即.png格式))
x_test_dir = os.path.join(DATA_DIR, 'test', 'img')
y_test_dir = os.path.join(DATA_DIR, 'test', 'label')
img_folder_path = x_test_dir # 要預測的圖片資料夾路徑

save_pred_img_folder_path = r'prediction'+ "/"+train_time # 儲存預測圖片的根目錄資料夾路徑
if not os.path.isdir(save_pred_img_folder_path):
    os.makedirs(save_pred_img_folder_path)
    
#增加預測對照圖的顯示位置1110831 add by charley
save_compare_img_folder_path = r'compare_output'+ "/"+train_time # 儲存預測對照圖的根目錄資料夾路徑 
if not os.path.isdir(save_compare_img_folder_path):
    os.makedirs(save_compare_img_folder_path)
#增加預測對照圖的顯示位置1110831 add by charley    
    
model_name = config["model"] # 使用的模型名稱 (Unet/FPN/Linknet/PSPNet)
backbone_name = config["BACKBONE"]# 使用的backbone名稱
num_classes = config["num_classes"] # 模型預測的類別數量, 此數值要+1(for unlabel)
color_list = [
                [0,240,255],#Alveolar_bone
                [65,127,0],#'Caries',
                [0,0,255],#Crown
                [113,41,29],#Dentin
                [122,21,135],#Enamel
                [0,148,242],#Implant
                [4,84,234],#Mandibular_alveolar_nerve
                [0,208,178],#Maxillary_sinus
                [52,97,148],#Periapical_lesion
                [121,121,121],#Post_and_core
                [212,149,27],#Pulp
                [206,171,255],#Restoration
                [110,28,216],#Root_canal_filling
                ]
                              # 針對資料前處理的correct_label清單, 依序填入想要顯示的BGR色碼 (註:是BGR不是RGB), background顏色不用填.
data_height=config["data_height"]
data_width=config["data_width"]

# MODEL=model_name
# if MODEL == 'PSPNet':
#     data_height=384
#     data_width=384
    # tf.compat.v1.disable_eager_execution()

change directory 配合使用使 os.getcwd() 可正確顯示Start
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/C_pred
change directory 配合使用使 os.getcwd() 可正確顯示End


In [8]:
ckpt_folder_path = config["save_ckpt_Folder"] # 存放model weight的資料夾路徑
# ckpt_folder_path="F:\\SurgeryAnalytics\\AI_Cases\\Dentistry/1_Linknet_efficientnetb7_0_1/B_AI_training/ckpt/20220712171222"
print(ckpt_folder_path)
config["save_pred_img_folder_path"]  = f"{cwd}/{save_pred_img_folder_path}"
#save_compare_img_folder_path
config["save_compare_img_folder_path"]  = f"{cwd}/{save_compare_img_folder_path}"
print(save_compare_img_folder_path)   #增加預測對照圖的顯示位置1110831 add by charley
config["color_list"] = color_list
AI_Data_jsonFile = Jsonname()
with open(AI_Data_jsonFile,"w") as f:
    json.dump(config, f)

/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/B_AI_training/ckpt/20230813005951
compare_output/20230813070234
AI_Data_jsonFile : == /media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/Dentistry_AI_ZERO_New_1120808_500_linknet_config.json


In [9]:
import os.path as path   # add by charley 1110830
import json              # add by charley 1110830
import operator          # add by charley 1110830
def ckptname():
    #print(ckpt_folder_path)
    dic = {}
    ckptnameKV = {}  #-------new20221108-yita-------- modify by charley 1120116
    ckptFolderPathListNEW = []
    for i in os.listdir(ckpt_folder_path):
        try:
            if i.split('.')[1]=='xlsx' or i.split('.')[1]=='pptx' or i.split('.')[1]=='ipynb_checkpoints'or i.split('.')[1]=='json' or i.split('.')[3]=='xlsx#' :
                print('Delete_list_name:'+i)   #20230505-yita-增加排除項目 json h5 xlsx#---------------
            else :
                ckptFolderPathListNEW.append(i)
        except:
            ckptFolderPathListNEW.append(i) 
            
    print(ckptFolderPathListNEW)
    #for ckptFileName in self.ckptFolderPathListNEW: #os.listdir(ckptFolderPath):
    for ckptFileName in ckptFolderPathListNEW: #os.listdir(ckpt_folder_path):
         #print(ckptFileName.split('_')[3]) # 列示全部的ckpt值
        k = ckptFileName.split('_')[2].strip() # key trim
        dic[k] = ckptFileName.split('_')[3].strip()  # value trim
    print(dic)  # 列示全部的ckpt值    
    json_object = json.dumps(dic, indent=4)
    max_keys = [key for key, value in dic.items() if value == max(dic.values())]
    print(max_keys[0]) #如為多個找出其中一個即可
    print("best solution ckpt-{} !".format(max_keys[0]))
    ckpname=max_keys[0] #存入自訂變數
    #return max_keys[0]
    print(list(dic.keys()))  
    #--------------------------new20221108-yita----------------modify charley 1120116
    ckptnameVal=list(dic.values()) [list (dic.keys()).index (max_keys[0])]   
    ckptnameKV[0]=ckpname
    ckptnameKV[1]=ckptnameVal
    # ckpnameVal=dic.values().index (max_keys[0])
    print(ckptnameVal)
    return ckptnameKV    #--------------------------new20221108-yita----------------

# AI_Cases/Dentistry2/C_pred/compare_output
# AI_Cases/Dentistry2/C_pred/labels
import shutil
def delfolder_compare_output():
    pathTeststr = AI_Path +"/C_pred/compare_output"
    print(pathTeststr)
    #pathTest = r pathTeststr
    #os.remove(path)
    try:
        shutil.rmtree(pathTeststr)
    except OSError as e:
        print(e)
    else:
        print("The directory is deleted successfully")
    
def delfolder_compare_labels():
    pathTeststr = AI_Path +"/C_pred/labels"
    print(pathTeststr)
    #pathTest = r pathTeststr
    try:
        shutil.rmtree(pathTeststr)
    except OSError as e:
        print(e)
    else:
        print("The directory is deleted successfully")
    

In [10]:
basic_parameters = [img_folder_path, save_pred_img_folder_path]
model_parameters = [model_name, backbone_name, num_classes, color_list, ckpt_folder_path]
print(basic_parameters)
print(model_parameters)

['/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/2_split/test/img', 'prediction/20230813070234']
['Linknet', 'efficientnetb7', 15, [[0, 240, 255], [65, 127, 0], [0, 0, 255], [113, 41, 29], [122, 21, 135], [0, 148, 242], [4, 84, 234], [0, 208, 178], [52, 97, 148], [121, 121, 121], [212, 149, 27], [206, 171, 255], [110, 28, 216]], '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/B_AI_training/ckpt/20230813005951']


# 模型預測

In [11]:
# pip install pipreqs



In [12]:
import os #1120509
from time import time
begin = time()
PredFolderImg(basic_parameters, model_parameters,data_height,data_width)
end = time()
print(end - begin)

TF version: 2.2.0.
Keras version: 2.3.0-tf.

/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/B_AI_training/ckpt/20230813005951
['Linknet_efficientnetb7_n31_0.833_model.h5', 'Linknet_efficientnetb7_n57_0.833_model.h5', 'Linknet_efficientnetb7_n61_0.832_model.h5']
n31
n57
n61
{'n31': '0.833', 'n57': '0.833', 'n61': '0.832'}
best solution ckpt-{
    "n31": "0.833",
    "n57": "0.833",
    "n61": "0.832"
} !
n31
best solution ckpt-n31 !
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/B_AI_training/ckpt/20230813005951
prediction/20230813070234/pred-Linknet_efficientnetb7_n31
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/B_AI_training/ckpt/20230813005951/Linknet_efficientnetb7_n31_0.833_model.h5
best solution path
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_

In [13]:
# dirPath = os.getcwd()
# AI_Data_jsonFile=AI_Path+'/B_AI_training/Dentistry_config.json' #modify by  charley 1110830
# # save_model_path = dirPath + "/ckpt"
# print(ckptname())
# config["ckptname"] = ckptname()  #ckptname best solution
# with open(AI_Data_jsonFile,"w") as f:
#     json.dump(config, f)

In [14]:

# jsonname


# dirPath = os.getcwd()
# AI_Data_jsonFile=AI_Path+'/B_AI_training/Dentistry_config.json' #modify by  charley 1110830
# print(ckptname())    #--------new20221108-yita-  #modify by  charley 1120116
ckptnameKV=ckptname()          
config["ckptname"] = ckptnameKV[0]  #ckptname best solution
config["ckptnameVal"] = ckptnameKV[1]  #ckptname best solution
# with open(AI_Data_jsonFile,"w") as f:
#     json.dump(config, f)
# config

['Linknet_efficientnetb7_n31_0.833_model.h5', 'Linknet_efficientnetb7_n57_0.833_model.h5', 'Linknet_efficientnetb7_n61_0.832_model.h5']
{'n31': '0.833', 'n57': '0.833', 'n61': '0.832'}
n31
best solution ckpt-n31 !
['n31', 'n57', 'n61']
0.833


In [15]:
AI_Data_jsonFile = Jsonname()
print(AI_Data_jsonFile )   #注意等等跑AI需要過來複製一下路徑
with open(AI_Data_jsonFile , 'w') as f:
     json.dump(config, f)
# config


AI_Data_jsonFile : == /media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/Dentistry_AI_ZERO_New_1120808_500_linknet_config.json
/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/Dentistry_AI_ZERO_New_1120808_500_linknet_config.json


In [16]:
config

{'surgeryName': 'Dentistry',
 'label_folder_path': '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/0_ori',
 'trans_folder_path': '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/1_trans',
 'split_folder_path': '/media/ai/disk1/notebook/jupyter_work/test1/SurgeryAnalytics/AI_Cases/Dentistry_AI_ZERO_New_1120808_500_linknet/0_Data/2_split',
 'correct_label': ['Alveolar_bone',
  'Caries',
  'Crown',
  'Dentin',
  'Enamel',
  'Implant',
  'Mandibular_alveolar_nerve',
  'Maxillary_sinus',
  'Periapical_lesion',
  'Post_and_core',
  'Pulp',
  'Restoration',
  'Root_canal_filling'],
 'detect_label_list': ['Alveolar_bone',
  'Caries',
  'Crown',
  'Dentin',
  'Enamel',
  'Implant',
  'Mandibular_alveolar_nerve',
  'Maxillary_sinus',
  'Periapical_lesion',
  'Post_and_core',
  'Pulp',
  'Restoration',
  'Root_canal_filling'],
 'Data_total_excel': '/media/

In [17]:
#修改為不需刪除 並増加日期年月日時間的資料夾 區隔即不會有重複的狀況 1110831 by charley
#delfolder_compare_output()
#delfolder_compare_labels()

In [18]:
#pip list